# Quantitative Momentum Strategy

Here we focus on desiging a strategy that allows us to focus on the stocks that have increased the most


In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'


## Importing Our List of Stocks

This is simply importing the list of stocks

In [178]:
stocks = pd.read_csv('sp_500_stocks-Copy1.csv')
stocks


,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
499,YUM
500,ZBH
501,ZBRA
502,ZION


In [179]:
#imported the api token from my secret file


from secrety import IEX_CLOUD_API_TOKEN



Testing to make sure the APi work by looking for one stock

In [180]:
symbol = 'AAPl'
api_url= f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data


{'companyName': 'Apple Inc',
 'marketcap': 2885604866228,
 'week52high': 191.03,
 'week52low': 120.99,
 'week52highSplitAdjustOnly': 191.8,
 'week52lowSplitAdjustOnly': 118.41,
 'week52change': 0.3550966581244424,
 'sharesOutstanding': 17028843009,
 'float': 0,
 'avg10Volume': 99252163,
 'avg30Volume': 105189408,
 'day200MovingAvg': 154.75,
 'day50MovingAvg': 176.93,
 'employees': 151073,
 'ttmEPS': 11.29,
 'ttmDividendRate': 0.8983518376170989,
 'dividendYield': 0.005111200101373454,
 'nextDividendDate': '',
 'exDividendDate': '2021-11-02',
 'nextEarningsDate': '2022-01-21',
 'peRatio': 15.362340115405082,
 'beta': 1.3489248047423381,
 'maxChangePercent': 65.98169558016112,
 'year5ChangePercent': 5.36111294640668,
 'year2ChangePercent': 1.2790769964887618,
 'year1ChangePercent': 0.3136994864664746,
 'ytdChangePercent': -0.03040931432369886,
 'month6ChangePercent': 0.19780535757428974,
 'month3ChangePercent': 0.2132662013539224,
 'month1ChangePercent': -0.04148688106966931,
 'day30Chan

We attempt to begin with parsing

In [181]:
data['year1ChangePercent']

0.3136994864664746

here we use a function from stack over flow that allows us to split lists into chunk.

proper documentation and credit is given

In [2]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])



my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

NameError: name 'stocks' is not defined

this code cell represents us saving our stocks to a dataframe

In [183]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,153.77,0.154293,N/A
1,AAL,19.63,0.243711,N/A
2,AAP,240.98,0.408349,N/A
3,AAPL,176.47,0.312019,N/A
4,ABBV,142.98,0.340052,N/A
...,...,...,...,...
499,YUM,133.58,0.26723,N/A
500,ZBH,132.73,-0.200431,N/A
501,ZBRA,560.93,0.33316,N/A
502,ZION,73.47,0.467721,N/A




Here we simply just looked for the fastest growing stocks and sorted them that way


In [215]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,80.52,2.371672,N/A
1,DVN,50.20,1.889136,N/A
2,F,24.41,1.688288,N/A
3,MRO,19.23,1.439881,N/A
4,MCHP,88.33,1.335997,N/A
5,FTNT,328.03,1.21682,N/A
6,FANG,126.31,1.211363,N/A
7,NVDA,284.95,1.114399,N/A
8,NUE,113.02,1.063976,N/A
9,SPG,168.98,0.963663,N/A


Calculating the number of shares to buy based on user input of potential portfolio size

In [216]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:100000000000
100000000000


In [217]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,80.52,2.371672,24838549
1,DVN,50.20,1.889136,39840637
2,F,24.41,1.688288,81933633
3,MRO,19.23,1.439881,104004160
4,MCHP,88.33,1.335997,22642363
5,FTNT,328.03,1.21682,6097003
6,FANG,126.31,1.211363,15834059
7,NVDA,284.95,1.114399,7018775
8,NUE,113.02,1.063976,17695983
9,SPG,168.98,0.963663,11835720


Here I improved on the original design by  focusing on what firmas called high quality momentum stocks.

High quality moement stocks tend to show steady and sustained performance over long periods of time while low quality stocks only have once in a while surges

We identified these high quality momentums by focusing on stocks with the highest percentiles of 1-month price returns, 3-month price returns, 6-month price returns, 1-year price returns



In [225]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:

    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                    'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,148.14,N/A,0.154062,N/A,-0.030474,N/A,-0.048805,N/A,-0.071712,N/A,N/A
1,AAL,19.56,N/A,0.244041,N/A,-0.10368,N/A,-0.066352,N/A,0.043676,N/A,N/A
2,AAP,243.51,N/A,0.40187,N/A,0.111506,N/A,0.100954,N/A,-0.020878,N/A,N/A
3,AAPL,177.37,N/A,0.321661,N/A,0.197783,N/A,0.21237,N/A,-0.040647,N/A,N/A
4,ABBV,140.00,N/A,0.336705,N/A,0.202468,N/A,0.24289,N/A,0.089837,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,136.16,N/A,0.268607,N/A,0.140165,N/A,0.086097,N/A,-0.003141,N/A,N/A
500,ZBH,132.18,N/A,-0.203202,N/A,-0.210609,N/A,-0.136647,N/A,0.019963,N/A,N/A
501,ZBRA,565.13,N/A,0.328678,N/A,-0.022787,N/A,0.076149,N/A,-0.131989,N/A,N/A
502,ZION,73.27,N/A,0.477675,N/A,0.352382,N/A,0.107423,N/A,0.104783,N/A,N/A


we use these metrics for our dataframe

 `One-Year Price Return`
 `Six-Month Price Return`
 `Three-Month Price Return`
 `One-Month Price Return`



In [235]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0
for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,148.14,N/A,0.154062,0.382937,-0.030474,0.244048,-0.048805,0.142857,-0.071712,0.119048,N/A
1,AAL,19.56,N/A,0.244041,0.521825,-0.10368,0.121032,-0.066352,0.119048,0.043676,0.613095,N/A
2,AAP,243.51,N/A,0.40187,0.753968,0.111506,0.642857,0.100954,0.625,-0.020878,0.315476,N/A
3,AAPL,177.37,N/A,0.321661,0.666667,0.197783,0.835317,0.21237,0.930556,-0.040647,0.22619,N/A
4,ABBV,140.00,N/A,0.336705,0.68254,0.202468,0.845238,0.24289,0.956349,0.089837,0.835317,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,136.16,N/A,0.268607,0.579365,0.140165,0.706349,0.086097,0.585317,-0.003141,0.384921,N/A
500,ZBH,132.18,N/A,-0.203202,0.031746,-0.210609,0.03373,-0.136647,0.039683,0.019963,0.5,N/A
501,ZBRA,565.13,N/A,0.328678,0.672619,-0.022787,0.265873,0.076149,0.547619,-0.131989,0.03373,N/A
502,ZION,73.27,N/A,0.477675,0.84127,0.352382,0.96627,0.107423,0.64881,0.104783,0.894841,N/A



We calculate our HQM score and use it to filter the stocks 
the hqm score is the arithmetic mean of the 4 momentum percentile scores. THe harmonic mean might have been a better choice here


In [238]:
from statistics import mean
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row,f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,148.14,N/A,0.154062,0.382937,-0.030474,0.244048,-0.048805,0.142857,-0.071712,0.119048,0.222222
1,AAL,19.56,N/A,0.244041,0.521825,-0.10368,0.121032,-0.066352,0.119048,0.043676,0.613095,0.34375
2,AAP,243.51,N/A,0.40187,0.753968,0.111506,0.642857,0.100954,0.625,-0.020878,0.315476,0.584325
3,AAPL,177.37,N/A,0.321661,0.666667,0.197783,0.835317,0.21237,0.930556,-0.040647,0.22619,0.664683
4,ABBV,140.00,N/A,0.336705,0.68254,0.202468,0.845238,0.24289,0.956349,0.089837,0.835317,0.829861
...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,136.16,N/A,0.268607,0.579365,0.140165,0.706349,0.086097,0.585317,-0.003141,0.384921,0.563988
500,ZBH,132.18,N/A,-0.203202,0.031746,-0.210609,0.03373,-0.136647,0.039683,0.019963,0.5,0.15129
501,ZBRA,565.13,N/A,0.328678,0.672619,-0.022787,0.265873,0.076149,0.547619,-0.131989,0.03373,0.37996
502,ZION,73.27,N/A,0.477675,0.84127,0.352382,0.96627,0.107423,0.64881,0.104783,0.894841,0.837798


we just sort the stocks by their HQM

In [248]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)

hqm_dataframe = hqm_dataframe[:100]

hqm_dataframe 



,level_0,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,0,174,F,25.15,N/A,1.739015,0.996032,0.678854,0.994048,0.598112,1.0,0.112672,0.920635,0.977679
1,1,148,DVN,51.40,N/A,1.891,0.998016,0.783223,0.996032,0.241146,0.954365,0.122509,0.936508,0.97123
2,2,98,CMA,100.04,N/A,0.668173,0.952381,0.436362,0.986111,0.168249,0.878968,0.165291,0.978175,0.948909
3,3,275,LB,82.40,N/A,2.363433,1.0,0.823733,0.998016,0.222206,0.944444,0.081909,0.803571,0.936508
4,4,407,SCHW,96.10,N/A,0.577975,0.904762,0.319976,0.952381,0.189421,0.90873,0.139344,0.956349,0.930556
5,5,271,KR,49.27,N/A,0.59292,0.916667,0.307268,0.94246,0.260558,0.968254,0.101729,0.878968,0.926587
6,6,42,APA,34.00,N/A,0.531147,0.878968,0.44619,0.988095,0.217847,0.940476,0.103672,0.888889,0.924107
7,7,262,KEY,27.32,N/A,0.513269,0.871032,0.336524,0.960317,0.162089,0.861111,0.144366,0.96627,0.914683
8,8,482,WFC,58.34,N/A,0.724098,0.962302,0.277827,0.920635,0.158903,0.85119,0.106252,0.900794,0.90873
9,9,365,PFE,57.27,N/A,0.593165,0.918651,0.452443,0.990079,0.338351,0.984127,0.066673,0.732143,0.90625


Calculated the number of shares based on the portfolio size

In [249]:
portfolio_input()

Enter the value of your portfolio:10000000


In [252]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
    
hqm_dataframe

,level_0,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,0,174,F,25.15,7952,1.739015,0.996032,0.678854,0.994048,0.598112,1.0,0.112672,0.920635,0.977679
1,1,148,DVN,51.40,3891,1.891,0.998016,0.783223,0.996032,0.241146,0.954365,0.122509,0.936508,0.97123
2,2,98,CMA,100.04,1999,0.668173,0.952381,0.436362,0.986111,0.168249,0.878968,0.165291,0.978175,0.948909
3,3,275,LB,82.40,2427,2.363433,1.0,0.823733,0.998016,0.222206,0.944444,0.081909,0.803571,0.936508
4,4,407,SCHW,96.10,2081,0.577975,0.904762,0.319976,0.952381,0.189421,0.90873,0.139344,0.956349,0.930556
5,5,271,KR,49.27,4059,0.59292,0.916667,0.307268,0.94246,0.260558,0.968254,0.101729,0.878968,0.926587
6,6,42,APA,34.00,5882,0.531147,0.878968,0.44619,0.988095,0.217847,0.940476,0.103672,0.888889,0.924107
7,7,262,KEY,27.32,7320,0.513269,0.871032,0.336524,0.960317,0.162089,0.861111,0.144366,0.96627,0.914683
8,8,482,WFC,58.34,3428,0.724098,0.962302,0.277827,0.920635,0.158903,0.85119,0.106252,0.900794,0.90873
9,9,365,PFE,57.27,3492,0.593165,0.918651,0.452443,0.990079,0.338351,0.984127,0.066673,0.732143,0.90625


here set up our xlsx file to print our recommendatiosn

In [256]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy', index = False)

In [257]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [258]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

saving our recommendations

In [259]:
writer.save()